In [3]:
from functools import reduce
import gc
import numpy as np

"""
STVM 계산(3차로 점선) + 지상부 추가 _ 변곡점 계산
    집계시간 : 1분
    분석시간 1800~6900 -> 6900까지 데이터 수집 후 5400으로 가공
    검지기 : 280개
"""
import pandas as pd

import os


# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 6900
collection_time = 60

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
######## 검지기 #############
# 지상부_진입 검지기(결과 데이터는 지상부_진입의 6번째 데이터 부터 들어가므로 해당 검지기를 선정함)
enter_line = 6

# 본선부 검지기
main_line = 140

# 지상부_진출 검지기
exit_line = 275
############################


######## 램프 ###############
# 유입램프
input_ramp = 59

# 유출램프
output_ramp = 61
############################

######## 구간 ###############
# 진입부_지상
entry_point = [i for i in range(6,11)]

# 본선부
middle_point = [i for i in range(41,241)]

# 진출부_ 지상
exit_point = [i for i in range(271,276)]
############################

###################################################################################################################

# 함수 모음
###################################################################################################################

# 평균속도
def speed_mean(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )

    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)

    return speed_mean_df

# 밀도
def density_mean(original_df):
    copy_df = original_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 가중 중차량 비율
def heavy_rate(original_df):
    copy_df = original_df.copy()
    # 진입부 검지기 선정
    measurement = enter_line

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup", "New_Measurement"],
        how="left"
    )

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 진입 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()

    # 실측용량 C
    max_capacity = 6600
    entry_saturation_df = (
        # 진입부에서 유입된 교통량이므로 진입부 중 한 개의 검지기를 선정하여 측정
        copy_df[copy_df["New_Measurement"] == enter_line]
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df


# 램프 유출입 비율
def rfr_rate(original_df):
    """
    1. 본선 1개의 검지기 data 수집(New_Measurement = 60)
    2. 위의 검지기 앞, 뒤 검지기 data (59 / 61)
    3. RFR 연산
    """
    main_df = (
        original_df[original_df["New_Measurement"] == main_line] # 60
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="main_line")
    )
    input_df = (
        original_df[original_df["New_Measurement"] == input_ramp] # 59
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="input_ramp")
    )
    output_df = (
        original_df[original_df["New_Measurement"] == output_ramp] # 61
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="output_ramp")
    )
    #main_df["RFR"] = (input_df["input_ramp"] + output_df["output_ramp"])/main_df["main_line"]
    # 유출램프 구현이 안 됐기 때문에 0값으로 fix
    main_df["RFR"] = 0
    return main_df

# 유출 정상성 비율
def output_normality(original_df):
    copy_df = original_df.copy()
    # 6번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    entry_df = copy_df[copy_df["New_Measurement"] == enter_line][["VehNo", "t(Entry)"]]

    # 265번 검지기(지상부_진출)에 들어온 차량의 번호, 시간
    exit_df = copy_df[copy_df["New_Measurement"] == exit_line][["VehNo", "t(Entry)"]]

    # 차량 번호로 그룹화 후 시간의 최솟값(중복제거)
    entry_first = (
        entry_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()  # Series → DataFrame
        .rename(columns={"t(Entry)": "t_entry"})
    )

    exit_first = (
        exit_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()
        .rename(columns={"t(Entry)": "t_exit"})
    )

    # 지연시간
    merged = pd.merge(entry_first, exit_first, on="VehNo", how="inner")
    merged["delay_sec"] = merged["t_exit"] - merged["t_entry"]
    merged = merged[merged["delay_sec"] >= 0]  # 음수 제거

    # 지연시간(중앙값) → lag_bins
    if len(merged) and np.isfinite(np.nanmedian(merged["delay_sec"])): # delay_sec의 값이 유효하면
        lag_bins = int(round(np.nanmedian(merged["delay_sec"]) / collection_time)) # 단위시간으로 나눴을 때의 중간값 => 3(900초) => 진입한 차량이 진출을 통과하는데 평균 900초가 걸림
    else:
        lag_bins = 0  # 데이터 부족 시 동시간 매칭

    # TimeGroup별 진입/유출 카운트 집계
    entry_count = (original_df[original_df["New_Measurement"] == enter_line]
                .groupby("TimeGroup").size().reset_index(name="Q_in"))
    exit_count  = (original_df[original_df["New_Measurement"] == exit_line]
                .groupby("TimeGroup").size().reset_index(name="Q_out"))

    merged_counts = pd.merge(entry_count, exit_count, on="TimeGroup", how="left")

    # Q_out을 지연 시간만큼 shift
    merged_counts["Q_out_shift"] = merged_counts["Q_out"].shift(-lag_bins)

    # 1−F(outrate) 계산
    merged_counts["F(outrate)"] = (merged_counts["Q_out_shift"] / merged_counts["Q_in"]).fillna(0)
    merged_counts["1-F(outrate)"] = 1 - merged_counts["F(outrate)"]
    return merged_counts


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):

    # TimeGroup 기준으로  Merge
    merged_df = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "New_Measurement", "rate"]], on=["TimeGroup", "New_Measurement"])
    .merge(entry_saturation_df[["TimeGroup", "Phi_진입"]], on="TimeGroup")
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "1-F(outrate)"]], on="TimeGroup")
    )

    merged_df["StartTime"] = merged_df["TimeGroup"].str.split("~").str[0].astype(int)
    merged_df = merged_df[merged_df["StartTime"] < 5400]

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["1-F(outrate)"]
    )
    merged_df = modify_frame(merged_df)
    return merged_df


def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 평균
    mean_stvm = copy_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = copy_df["STVM"].std(skipna=True)

    # Z-Score 계산
    copy_df["Z-Score"] = (copy_df["STVM"] - mean_stvm) / std_stvm
    z_max = copy_df["Z-Score"].max()
    z_min = copy_df["Z-Score"].min()

    copy_df["환산점수"] = copy_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(copy_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    modify_df = df.copy()

    modify_df = modify_df[modify_df["New_Measurement"] != 276]
    modify_df["New_Measurement"] = modify_df.groupby("TimeGroup").cumcount() + 1
    return modify_df

def calculate_avg(df):

    def avg_range(df, low, high):
        cols = [c for c in df.columns if low <= c <= high]
        return df[cols].mean(axis=1) if cols else np.nan

    avg_df = pd.DataFrame(index=df.index)
    avg_df["지상부_진입"] = avg_range(df, -5, -1)
    avg_df["진입부"] = avg_range(df, 1, 30)
    avg_df["본선부"] = avg_range(df, 31, 230)
    avg_df["진출부"] = avg_range(df, 231, 260)
    avg_df["지상부_진출"] = avg_range(df, 261, 265)

    return avg_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "1-F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    return copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = r"C:\VISSIM_Workspace\network_version2.14.0"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()


for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, collection_time)
                labels = [f"{start}~{start+collection_time}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)
                measurement_list = [
                    # 지상부_진입
                    6,
                    # 지상부-진입부
                    9, 10, 11,
                    # 진입부 - 본선부
                    39, 40, 41,
                    # 본선부
                    140,
                    # 본선부 - 진출부
                    239,240,241,
                    # 진출부 - 지상부
                    269, 270,271,
                    # 지상부_진출
                    275, 276
                ]
                original_df = original_df[original_df["New_Measurement"].isin(measurement_list)]

                # 평균속도
                speed_df = speed_mean(original_df)

                # 밀도
                density_df = density_mean(speed_df)

                # 가중 중차량 비율
                heavy_df = heavy_rate(original_df)

                # 진입 포화도
                entry_saturation_df = entry_saturation(original_df)

                # 램프 유출입 비율
                rfr_df = rfr_rate(original_df)

                # 진출 정상성
                normality_df = output_normality(original_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)
                display(stvm_df)
                #save_to_excel(z_score_df, folder_path, "환산점수", i)

                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                display("z_score_df : ", z_score_df)
                #save_to_excel(z_score_df, folder_path, "환산점수", i)

                # 변수별 TimeGroup별 평균
                #variable_time_df = variable_timegroup_avg(stvm_df)
                #save_to_excel(variable_time_df, folder_path, "변수별 평균", i)

                # 변수별 전체 평균
                #variable_total_df = variable_total_avg(variable_time_df)
                #save_to_excel(variable_total_df, folder_path, "변수별 총 평균", i)

                # STVM 피봇
                #stvm_pivot_df = pivot_table(stvm_df, "STVM")
                #save_to_excel(stvm_pivot_df, folder_path, "STVM", i)

                # 속도, 밀도 전체 평균값
                #speed_density_df = speed_density_avg(density_df)
                #save_to_excel(speed_density_df, folder_path, "속도_밀도 평균값", i)

                # 속도 변화 피봇
                #delta_speed_pivot_df = pivot_table(speed_df, "delta_V", preprocess=modify_frame)
                #save_to_excel(delta_speed_pivot_df, folder_path, "속도변화량", i)

                # 밀도 변화 피봇
                #delta_density_pivot_df = pivot_table(density_df, "delta_K", preprocess=modify_frame)
                #save_to_excel(delta_density_pivot_df, folder_path, "밀도변화량", i)

                # 속도값 피봇
                #speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 가중평균속도
                #weighted_speed_df = weighted_avg_speed(original_df)
                #weighted_speed_df = pivot_table(weighted_speed_df, "Weighted_Avg_Speed", preprocess=modify_frame)
                #save_to_excel(weighted_speed_df, folder_path, "가중평균속도", i)
                # 메모리 정리
                del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()

,TimeGroup,New_Measurement,delta_V,delta_K,rate,Phi_진입,RFR,1-F(outrate),StartTime,STVM
0,1800~1860,1,-0.011745,0.128640,0.000000,0.047273,0,-0.038462,1800,0.125707
1,1800~1860,2,-0.000301,-0.034192,0.000000,0.047273,0,-0.038462,1800,-0.025682
2,1800~1860,3,-0.008032,0.044100,0.000000,0.047273,0,-0.038462,1800,0.044880
3,1800~1860,4,0.004660,-0.107606,0.000000,0.047273,0,-0.038462,1800,-0.094136
4,1800~1860,5,-0.010513,0.049495,0.000000,0.047273,0,-0.038462,1800,0.047793
...,...,...,...,...,...,...,...,...,...,...
954,5340~5400,11,-0.015493,0.269670,0.107143,0.054545,0,0.000000,5340,0.415866
955,5340~5400,12,-0.014263,-0.043500,0.057143,0.054545,0,0.000000,5340,0.053925
956,5340~5400,13,-0.001252,0.031595,0.060606,0.054545,0,0.000000,5340,0.145494
957,5340~5400,14,0.018130,-0.277799,0.088235,0.054545,0,0.000000,5340,-0.116889


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
TimeGroup,,,,,,,,,,,,,,,
1800~1860,71.434384,66.826778,70.225129,63.142233,70.268716,71.897243,70.661740,72.445347,68.780185,70.102916,53.110234,65.974241,63.005179,93.024914,68.790826
1860~1920,57.088798,56.200062,56.213380,54.481152,56.206205,54.343901,85.744778,55.370688,59.132599,54.733945,52.732751,60.469421,58.792826,65.970212,59.659609
1920~1980,90.750689,90.581394,91.752009,72.118268,89.924492,94.126605,63.794180,98.261896,89.343918,71.293794,99.770610,84.509433,83.150503,85.770693,84.359233
1980~2040,60.394603,56.532807,57.595080,67.136747,58.459975,58.460039,53.720695,63.361946,56.519951,72.117926,54.881156,57.599763,61.579412,57.420866,56.533778
2040~2100,71.393661,86.503333,71.896513,80.017432,72.381646,70.890882,85.998254,61.784443,64.320198,71.384323,97.758527,71.923292,71.941376,70.837703,80.704159
2100~2160,85.774369,81.885406,85.973091,89.261024,81.811045,87.371403,71.075386,85.990931,95.310241,84.276110,70.832838,91.499290,86.489168,84.982952,88.196158
2160~2220,95.515134,97.605439,97.589513,93.219779,95.824057,95.823533,95.010054,98.372763,95.313066,94.637151,93.774609,86.927540,89.897598,86.539781,93.820311
2220~2280,68.119168,59.488212,61.237587,71.140096,60.904892,62.558533,59.525070,70.527629,59.498062,62.344180,52.606731,71.739417,80.288990,70.234149,62.936837
2280~2340,54.647160,54.643247,54.643275,70.943433,62.968142,54.056463,56.625856,52.913276,56.191798,54.643494,95.748224,57.155980,53.514680,52.898748,56.553989


,TimeGroup,New_Measurement,delta_V,delta_K,rate,Phi_진입,RFR,1-F(outrate),StartTime,STVM
0,1800~1860,1,-0.011745,0.128640,0.000000,0.047273,0,0.076923,1800,0.241091
1,1800~1860,2,-0.000301,-0.034192,0.000000,0.047273,0,0.076923,1800,0.089703
2,1800~1860,3,-0.008032,0.044100,0.000000,0.047273,0,0.076923,1800,0.160264
3,1800~1860,4,0.004660,-0.107606,0.000000,0.047273,0,0.076923,1800,0.021249
4,1800~1860,5,-0.010513,0.049495,0.000000,0.047273,0,0.076923,1800,0.163178
...,...,...,...,...,...,...,...,...,...,...
954,5340~5400,11,-0.058074,0.364984,0.107143,0.054545,0,-0.100000,5340,0.368599
955,5340~5400,12,-0.012694,-0.071548,0.083333,0.054545,0,-0.100000,5340,-0.046363
956,5340~5400,13,0.025330,-0.054259,0.090909,0.054545,0,-0.100000,5340,0.016526
957,5340~5400,14,0.029582,-0.271549,0.093750,0.054545,0,-0.100000,5340,-0.193671


'z_score_df : '

...

'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
TimeGroup,,,,,,,,,,,,,,,
1800~1860,55.684277,54.835287,53.365120,55.930792,53.078797,56.735440,56.000546,53.931274,54.052897,54.720819,53.014730,54.120546,56.097574,68.687749,59.321636
1860~1920,62.570095,65.594313,68.144181,70.954271,63.856278,60.716506,58.814872,71.867210,61.640228,60.352675,69.566956,58.637681,61.124738,57.750682,56.854232
1920~1980,70.796561,66.031822,65.552807,60.660081,70.981016,68.389555,97.512881,52.589661,68.198084,71.590417,82.247147,80.936670,70.378243,71.797120,70.001063
1980~2040,55.552355,57.867265,59.179495,55.097985,63.452189,68.055630,67.230353,70.236873,58.592847,57.249626,81.909240,58.027822,64.320643,58.459296,58.755096
2040~2100,67.024106,67.278420,64.627831,62.063096,64.139976,65.872705,70.074560,57.948777,70.999899,70.117597,87.118494,67.499591,69.925808,70.545526,68.731244
2100~2160,71.071005,70.196924,70.905226,71.004509,67.524369,66.950445,57.594650,95.158558,70.332600,70.626231,60.863815,71.410053,70.915865,69.634487,70.436178
2160~2220,72.230389,85.869364,81.509413,91.133794,80.969304,71.916026,71.176262,94.937902,72.281926,82.450527,70.295571,81.244301,85.179126,92.536437,81.865768
2220~2280,93.145862,89.289347,91.979803,86.499146,93.136545,93.105112,98.213386,68.934429,94.724111,93.041510,98.378193,88.479447,88.968122,90.551640,90.533601
2280~2340,71.819399,71.919383,71.848448,68.013409,70.861273,71.527786,67.917963,99.015381,71.388885,71.193020,67.200453,81.338781,82.763882,71.676111,80.154519


,TimeGroup,New_Measurement,delta_V,delta_K,rate,Phi_진입,RFR,1-F(outrate),StartTime,STVM
0,1800~1860,1,0.002442,0.039714,0.140845,0.129091,0,0.211268,1800,0.523360
1,1800~1860,2,0.006365,-0.006325,0.148649,0.129091,0,0.211268,1800,0.489047
2,1800~1860,3,0.007647,-0.047822,0.108108,0.129091,0,0.211268,1800,0.408292
3,1800~1860,4,-0.063408,0.142891,0.112676,0.129091,0,0.211268,1800,0.532517
4,1800~1860,5,-0.011240,-0.041863,0.105263,0.129091,0,0.211268,1800,0.392520
...,...,...,...,...,...,...,...,...,...,...
954,5340~5400,11,-0.283299,0.880597,0.101449,0.136364,0,0.440000,5340,1.275111
955,5340~5400,12,0.046603,-0.106171,0.161290,0.136364,0,0.440000,5340,0.678086
956,5340~5400,13,0.402497,-0.303377,0.149425,0.136364,0,0.440000,5340,0.824909
957,5340~5400,14,0.237971,-0.192226,0.117647,0.136364,0,0.440000,5340,0.739755


'z_score_df : '

New_Measurement,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
TimeGroup,,,,,,,,,,,,,,,
1800~1860,84.138963,82.243814,71.873384,84.641296,71.637576,86.175735,84.770811,82.164372,82.429222,81.997400,98.618830,81.375589,90.089176,89.236843,80.005198
1860~1920,53.293068,54.710619,56.224182,60.258907,53.971078,52.514803,52.566634,61.133614,52.641218,52.866597,52.656128,52.866557,54.954150,52.907225,53.052812
1920~1980,62.378191,56.679593,56.364739,53.503936,63.000369,58.369751,94.390184,53.055883,57.757979,59.226717,98.643094,56.093392,59.763666,59.835225,57.493348
1980~2040,61.768444,65.970792,68.137863,60.997846,71.178358,80.468503,72.172714,72.153053,68.089804,68.299393,95.695328,82.191008,83.515466,71.399916,70.726296
2040~2100,70.728570,70.808544,70.042772,66.496989,69.457372,70.366487,71.710839,68.310505,70.368319,71.453663,97.596283,66.605498,80.316116,80.000931,71.568091
2100~2160,72.107371,71.098843,71.916093,72.030647,70.132321,69.815412,58.201362,93.992049,72.494325,71.756518,96.562245,72.159480,83.488996,80.052195,70.997798
2160~2220,60.458348,67.722799,62.798952,71.405619,62.303407,59.444389,57.242146,82.533795,61.161754,62.697318,68.845725,65.385857,69.462930,68.157184,62.195649
2220~2280,71.657048,69.710465,71.069849,65.887351,71.651905,71.634553,97.550133,55.130410,80.476110,71.587105,98.278725,67.366956,71.127998,72.002914,70.229891
2280~2340,72.306693,72.422057,72.340210,67.688274,71.215421,71.802933,70.055067,98.531400,71.239220,70.989645,90.808646,84.992882,86.575354,88.088215,81.267599
